In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Data aggregation").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/08 06:20:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
listings = spark.read.csv(
    "data/listings.csv.gz",
    header=True,
    inferSchema=True,
    sep=",",
    quote='"',
    escape='"',
    multiLine=True,
    mode="PERMISSIVE",
)

In [3]:
listings.groupby(listings.property_type).count().show(truncate=True)

+--------------------+-----+
|       property_type|count|
+--------------------+-----+
|Private room in l...|    2|
|Private room in loft|  154|
|Private room in e...|    2|
|       Entire chalet|    5|
|        Earthen home|    1|
|           Farm stay|    4|
|  Entire rental unit|40799|
|Shared room in ho...|   61|
|         Shared room|    1|
|Private room in c...| 3255|
|Room in boutique ...|  229|
|Private room in r...|    4|
|Room in bed and b...|   18|
|Private room in c...|   62|
|Private room in b...|   63|
|        Entire cabin|   50|
|   Entire guesthouse|  226|
|                 Hut|    4|
|Private room in n...|    4|
|  Entire guest suite|  175|
+--------------------+-----+
only showing top 20 rows


In [4]:
import pyspark.sql.functions as F

listings.groupBy(listings.property_type).agg(
    F.count("property_type").alias("count")
).orderBy("count", ascending=[False]).show(truncate=False)

+----------------------------------+-----+
|property_type                     |count|
+----------------------------------+-----+
|Entire rental unit                |40799|
|Private room in rental unit       |14573|
|Private room in home              |11826|
|Entire home                       |8938 |
|Entire condo                      |8438 |
|Private room in condo             |3255 |
|Entire serviced apartment         |1840 |
|Private room in townhouse         |1189 |
|Entire townhouse                  |1069 |
|Room in hotel                     |1042 |
|Private room in bed and breakfast |495  |
|Private room in guesthouse        |371  |
|Entire loft                       |339  |
|Room in boutique hotel            |229  |
|Entire guesthouse                 |226  |
|Entire guest suite                |175  |
|Private room in guest suite       |170  |
|Private room in loft              |154  |
|Private room in serviced apartment|154  |
|Private room                      |103  |
+----------

In [5]:
listings.groupBy(listings.property_type).agg(
    F.count("property_type").alias("count"), F.avg("review_scores_location")
).orderBy("count", ascending=[False]).show(truncate=False)


+----------------------------------+-----+---------------------------+
|property_type                     |count|avg(review_scores_location)|
+----------------------------------+-----+---------------------------+
|Entire rental unit                |40799|4.733095834431922          |
|Private room in rental unit       |14573|4.723925292814142          |
|Private room in home              |11826|4.701917440156306          |
|Entire home                       |8938 |4.726180264430801          |
|Entire condo                      |8438 |4.778066335437068          |
|Private room in condo             |3255 |4.7702538787023885         |
|Entire serviced apartment         |1840 |4.718870662460569          |
|Private room in townhouse         |1189 |4.762151898734181          |
|Entire townhouse                  |1069 |4.817585836909877          |
|Room in hotel                     |1042 |4.618212996389897          |
|Private room in bed and breakfast |495  |4.741178247734139          |
|Priva

In [6]:
reviews = spark.read.csv(
    "data/reviews.csv.gz",
    header=True,
    inferSchema=True,
    sep=",",
    quote='"',
    escape='"',
    multiLine=True,
    mode="PERMISSIVE",
)


In [7]:
for field in reviews.schema:
    print(field)

StructField('listing_id', LongType(), True)
StructField('id', LongType(), True)
StructField('date', DateType(), True)
StructField('reviewer_id', IntegerType(), True)
StructField('reviewer_name', StringType(), True)
StructField('comments', StringType(), True)


In [8]:
listings_reviews = listings.join(
    reviews, listings.id == reviews.listing_id, how="inner"
)

In [9]:
listings_reviews.groupBy("id").agg(F.count("id").alias("num_reviews")).show()

{"ts": "2025-09-08 07:18:14.126", "level": "ERROR", "logger": "DataFrameQueryContextLogger", "msg": "[AMBIGUOUS_REFERENCE] Reference `id` is ambiguous, could be: [`id`, `id`]. SQLSTATE: 42704", "context": {"file": "java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)", "line": "", "fragment": "col", "errorClass": "AMBIGUOUS_REFERENCE"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o160.agg.\n: org.apache.spark.sql.AnalysisException: [AMBIGUOUS_REFERENCE] Reference `id` is ambiguous, could be: [`id`, `id`]. SQLSTATE: 42704\n\tat org.apache.spark.sql.errors.QueryCompilationErrors$.ambiguousReferenceError(QueryCompilationErrors.scala:2163)\n\tat org.apache.spark.sql.catalyst.expressions.package$AttributeSeq.resolve(package.scala:356)\n\tat org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveChildren(LogicalPlan.scala:164)\n\tat org.apache.spark.sql.catalyst.analysis.ColumnResolutionHelper.$anonfun$resolveExpressionByP

AnalysisException: [AMBIGUOUS_REFERENCE] Reference `id` is ambiguous, could be: [`id`, `id`]. SQLSTATE: 42704

In [10]:
reviews_per_listing = (
    listings_reviews.groupBy(listings.id, listings.name)
    .agg(F.count(reviews.id).alias("num_reviews"))
    .orderBy("num_reviews", ascending=False)
    .show(truncate=False)
)

+--------+--------------------------------------------------+-----------+
|id      |name                                              |num_reviews|
+--------+--------------------------------------------------+-----------+
|47408549|Double Room+ Ensuite                              |1855       |
|30760930|Double Garden View room - London House Hotel***   |1682       |
|43120947|Private double room with en suite facilities      |1615       |
|19670926|Locke Studio Apartment at Leman Locke             |1436       |
|45006692|Budget Double Room In Colliers Hotel.             |1433       |
|1436172 |Cosy Double in Kings Cross Houseshare nr Eurostar |1195       |
|2126708 |London's best transport hub 5 mins walk! Safe too!|1122       |
|1436177 |En-suite Double in Kings Cross Houseshare Eurostar|1005       |
|47438714|KX Basic- Small Double- shared bathroom           |978        |
|3855375 |Double in Kings Cross Houseshare nr Eurostar      |973        |
|46233904|Superior Studio, avg size 23